In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from datetime import datetime
import csv

/Users/tanujdeshmukh/Desktop/Georgia Tech/Fall 2024/NonParametricDataAnalysis/Project/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
def fetch_monthly_data(symbol, api_key):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={symbol}&apikey={api_key}'
    response = requests.get(url)
    data = response.json()

    if "Monthly Time Series" in data:
        return data["Monthly Time Series"]
    else:
        raise ValueError(f"Error fetching data for {symbol}: {data.get('Error Message', 'Unknown error')}")


In [4]:
def filter_last_x_years(time_series_data,x):
    """
    Filter monthly time series data for the last 15 years.

    Args:
        time_series_data (dict): Raw time series data from Alpha Vantage.

    Returns:
        dict: Filtered data for the last 15 years.
    """
    filtered_data = {}
    current_year = datetime.now().year

    for date, metrics in time_series_data.items():
        year = int(date.split("-")[0])
        if current_year - x <= year <= current_year:
            filtered_data[date] = metrics

    return filtered_data


In [5]:
def save_to_csv(data, filename):
    """
    Save the stock data to a CSV file.

    Args:
        data (dict): Stock data to save.
        filename (str): Name of the CSV file.
    """
    with open(filename, mode='w') as file:
        writer = csv.writer(file)
        writer.writerow(["Date", "Open", "High", "Low", "Close", "Volume"])
        for date, metrics in sorted(data.items()):
            writer.writerow([date, metrics['1. open'], metrics['2. high'], metrics['3. low'], metrics['4. close'], metrics['5. volume']])
    print(f"Data saved to {filename}")


In [6]:
stock_symbols = ["IBM", "AAPL", "GOOGL", "MSFT"]

In [8]:
API_KEY = "YP0JT6U17TFGVUME"

In [10]:
for symbol in stock_symbols:
    try:
        print(f"Fetching monthly data for {symbol}...")
        monthly_data = fetch_monthly_data(symbol, API_KEY)
        last_15_years_data = filter_last_x_years(monthly_data,20)
        
        # Save to CSV
        filename = f"{symbol}_monthly_last_20_years.csv"
        save_to_csv(last_15_years_data, filename)
    
    except Exception as e:
        print(f"An error occurred while processing {symbol}: {e}")

Fetching monthly data for IBM...
Data saved to IBM_monthly_last_20_years.csv
Fetching monthly data for AAPL...
Data saved to AAPL_monthly_last_20_years.csv
Fetching monthly data for GOOGL...
Data saved to GOOGL_monthly_last_20_years.csv
Fetching monthly data for MSFT...
Data saved to MSFT_monthly_last_20_years.csv


In [12]:
def save_combined_csv(stock_data_list, symbols, filename):
    """
    Save combined stock data for multiple symbols into a single CSV.

    Args:
        stock_data_list (list): List of stock data dicts.
        symbols (list): List of stock symbols.
        filename (str): Name of the CSV file.
    """
    with open(filename, mode='w') as file:
        writer = csv.writer(file)
        writer.writerow(["Symbol", "Date", "Open", "High", "Low", "Close", "Volume"])
        for symbol, data in zip(symbols, stock_data_list):
            for date, metrics in sorted(data.items()):
                writer.writerow([symbol, date, metrics['1. open'], metrics['2. high'], metrics['3. low'], metrics['4. close'], metrics['5. volume']])
    print(f"Combined data saved to {filename}")

# Combine and save all stock data
all_data = [filter_last_x_years(fetch_monthly_data(symbol, API_KEY),20) for symbol in stock_symbols]
save_combined_csv(all_data, stock_symbols, "combined_stock_data_last_20_years.csv")


TypeError: filter_last_x_years() missing 1 required positional argument: 'x'